In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

In [ ]:
%aiida
%matplotlib notebook
import base64
import pandas as pd
import urllib.parse as urlparse
import ipywidgets as ipw
from IPython.display import display, clear_output
from aiidalab_widgets_base import viewer
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
uuid = urlparse.parse_qs(url.query)['uuid'][0]
workcalc = load_node(uuid=str(uuid))
ism = workcalc.outputs.output_parameters['isotherm']

isotherm = np.array([ism['pressure'], ism['loading_absolute_average'], ism['loading_absolute_dev']]).transpose()



enthalpy = np.array([ism['enthalpy_of_adsorption_average'], ism['enthalpy_of_adsorption_dev']]).transpose()

# Cleaning up.
to_delete = []
for i, p in enumerate(enthalpy):
    if None in p:
        to_delete.append(i)

if to_delete:
    enthalpy = np.delete(enthalpy, *to_delete, axis=0)
    isotherm = np.delete(isotherm, *to_delete, axis=0)

enth_av, enth_dev = zip(*enthalpy)
ism_p, ism_load, ism_dev = zip(*isotherm)


molecule_converter = {
    "co2": "CO2",
    "ch4": "CH4",
    "n2" : "N2",
    "h2o": "H2O",
    "h2" : "H2",
    "o2" : "O2",
}

# Detailed Report

## Structure

In [ ]:
display(viewer(workcalc.inputs.structure))

## Isotherm and Enthalpy plots

In [ ]:
if workcalc.outputs.output_parameters['is_porous']:
    
    # Adsorption
    fig, axs = plt.subplots(1,2, figsize=(9.9, 5))
    line1 = axs[0].errorbar(ism_p, ism_load, yerr=ism_dev, fmt='-or', ecolor='#f58d7f')
    axs[0].set_title("Isotherm")
    axs[0].grid(True)
    pressure_label = "Pressure [{}]".format(ism['pressure_unit'])
    axs[0].set_xlabel(pressure_label)
    loading_label = "{} loading [{}]".format(molecule_converter[workcalc.inputs['molecule'].value], ism['loading_absolute_unit'])
    axs[0].set_ylabel(loading_label)
    line_exp, = axs[0].plot([], [], '-ob')
    

    # Enthalpy
    line2 = axs[1].errorbar(enth_av, ism_load, xerr=enth_dev, fmt='-or')
    axs[1].grid(True)
    axs[1].set_title("Enthalpy of adsorption [{}]".format(ism['enthalpy_of_adsorption_unit']))
    enthalpy_label = "Enthalpy of adsorption [{}]".format(ism['enthalpy_of_adsorption_unit'])
    axs[1].set_xlabel(enthalpy_label)
    axs[1].set_ylabel(loading_label)
    min_value = min(np.array(enth_av)-np.array(enth_dev))
    max_value = max(np.array(enth_av)+np.array(enth_dev))
    axs[1].set_xlim([min_value-2,  max_value+2])
    
    
    # CSV
    dataf = pd.DataFrame([(key, ism_load[i], enth_av[i]) for i, key in enumerate(ism_p)],
                         columns=[pressure_label, loading_label, enthalpy_label])
    table = ipw.HTML('')
    payload = base64.b64encode(dataf.to_csv(index=False).encode()).decode()
    fname = 'data.csv'
    to_add = """Download data in csv format: <a download="{filename}"
    href="data:text/csv;base64,{payload}" target="_blank">{title}</a>"""
    table.value = to_add.format(filename=fname, payload=payload, title=fname)
    display(table)
    
else:
    print("The material is not porous. No isotherm produced.")

In [ ]:
text_plot = ipw.Textarea(
    value='',
    placeholder='Pressure [bar] loading [mol/kg]',
    description='Experimental isotherm:',
    disabled=False,
    layout={'width': "40%", 'height': '300px'},
    style = {"description_width": "initial"}
)

def on_apply(_=None):
    to_plot = []
    for line in text_plot.value.splitlines():
        to_plot.append(list(map(float, line.split())))
    res = np.array(to_plot).transpose()
    
    if res.size > 0:
        line_exp.set_data(res[0], res[1])
    else:
        line_exp.set_data([], [])
    fig.canvas.draw_idle()
     
 
plot_button = ipw.Button(description="Plot")
plot_button.on_click(on_apply)
box = ipw.VBox(children=[text_plot, plot_button])
accordion = ipw.Accordion(children=[box], selected_index=None)
accordion.set_title(0, 'Compare to experiment')
display(accordion)

## Computed properties

In [ ]:
display(viewer(workcalc.outputs.output_parameters))